This notebook sets up and runs a series of analysis stages that are dispatched over the compute nodes of a multiprocessing environment, either locally (using the `multiprocessing` Python module) or remotely onto a *Slurm*+*singularity* enabled computer cluster.

The analysis consists of processing regions of interest from two SPT data files.
Each region of interest is spatially segmented, and then DV inference is performed in the resulting space bins.

This can be done as a single stage, but in this example three stages are defined instead:

* a *tessellate* stage performs the segmentation,
* an *infer* stage runs the inference,
* and in between a *reload* stage is introduced to synchronize the multiple workspaces, aligning the state of the RWAnalyzer objects on the *.rwa* files generated or updated by the *tessellate* stage.

If the notebook is run at least until the `a.run()` cell, the corresponding *.ipynb* file is exported and run in other processes or worker nodes.
This implies that the notebook should be saved (*Save and Checkpoint*) before you *Restart & Run All*, if it has been modified.

The first notebook cells show how to set up the pipeline. The pipeline is actually launched at the `a.run()` cell of code, where `a` is the main `RWAnalyzer` object.
The notebook lines after the first call to the `run` method are never dispatched. Any second or third call to `run` would run the same initial part of the notebook.

# A simple *tessellate-and-infer* pipeline to resolve diffusivity and effective potential in space

The main difficulty in dispatching computations on remote hosts lies in locating the data files.
An approach that has been favored in the use cases developped as of version `0.5` consists of making all the paths absolute, possibly using the '~' placeholder of the home/user directory.

In [1]:
import os

wd = '~/' + os.path.relpath(os.getcwd(), os.path.expanduser('~')).replace('\\', '/')
wd

'~/github/TRamWAy/notebooks'

We set up an `RWAnalyzer` object with SPT data files, the corresponding files for regions of interest,
the segmentation to be applied to each ROI, and the inference procedure to be applied to each microdomain.

In [2]:
from tramway.analyzer import *

a                                 = RWAnalyzer()

a.spt_data.from_ascii_files(f'{wd}/data-examples/*.rpt.txt')
a.spt_data.localization_precision = 0.03

a.roi.from_ascii_files(suffix='roi') # => *.rpt-roi.txt

a.tesseller                       = tessellers.Hexagons

a.mapper.from_plugin('stochastic.dv')
a.mapper.diffusivity_prior        = 20
a.mapper.potential_prior          = 1
a.mapper.max_runtime              = 100 # in seconds; 100 seconds is much too short for a proper DV estimation, but convenient for a quick example
a.mapper.verbose                  = False
a.mapper.worker_count             = None if os.name == 'nt' else 4 # Windows OS is not fully supported yet

Let us make sure the paths are alright. The following two notebook cells are optional.

Of note, this notebook assumes the [introduction notebook](RWAnalyzer%20tour.ipynb) ran before, or at least its very first code cell, so that the input data are available.

In [3]:
a.spt_data.filepaths

['/home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-02-15ms.rpt.txt',
 '/home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-01-15ms.rpt.txt']

In [4]:
[ f.roi.filepath for f in a.spt_data ]

['/home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-02-15ms.rpt-roi.txt',
 '/home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-01-15ms.rpt-roi.txt']

Below are defined the different pipeline stages, using building blocks available in the `stages` module exported by the `tramway.analyzer` package.

In [5]:
a.pipeline.append_stage(stages.tessellate())
a.pipeline.append_stage(stages.reload())
a.pipeline.append_stage(stages.infer())

The declarative approach to setting a pipeline grants control over how to run the pipeline.

The aim of such a design consisted in making such inferences run on computer clusters, in particular Slurm- and singularity-enabled clusters, communicating with these clusters through an SSH connection.

At *Institut Pasteur* in Paris, France, the main computer cluster was baptised *Maestro*.
The `environments` module exported by the `tramway.analyzer` package features a `Maestro` predefined environment that can be used to set the `env` attribute of an `RWAnalyzer` object.

This environment object takes the local username to connect to the *Maestro* submit node. This can be overriden with the `env.username` attribute.

However, the Maestro cluster is accessible only over *Institut Pasteur*'s VPN or from the campus.

To make this notebook run in more different circumstances, we will use the `LocalHost` environment instead.
This environment operates in a similar fashion, but on the local computer, and does not involve any remote resource.

It actually offers a convenient way to test a pipeline before running the same pipeline on a computer cluster.

In [6]:
#a.env                             = environments.Maestro # works only over Institut Pasteur's VPN or on campus
a.env                             = environments.LocalHost # replacement so that the demo can work anywhere
a.env.worker_count                = 10

a.env.script                      = 'RWAnalyzer standard pipeline.ipynb'

If someday you may export this notebook as a regular Python script, you should consider using the `__file__` variable, as demonstrated in the following code cell, as a replacement for the last line above. Otherwise, you can safely delete the following cell:

In [7]:
try:
    a.env.script                  = __file__
except NameError:
    # in an IPython notebook, `__file__` is not defined and there is no standard way to get the notebook's name
    a.env.script                  = 'RWAnalyzer standard pipeline.ipynb'

The following cell is also optional. The default logging (or verbosity) level is `INFO`.

In [8]:
import logging

a.logger.setLevel(logging.DEBUG)

The `run` method launches the pipeline.
The workload is concentrated in the following code cell:

In [9]:
a.run()

working directory: /tmp/tmpo5pq5ptj
setup complete
running: jupyter nbconvert --to python "/home/flaurent/github/TRamWAy/notebooks/RWAnalyzer standard pipeline.ipynb" --stdout
initial dispatch done
jobs ready
submitting: /usr/bin/python3 /tmp/tmpo5pq5ptj/tmpocjtmzkl.py --working-directory="/tmp/tmpo5pq5ptj" --stage-index=0 --source="/home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-02-15ms.rpt.txt"
submitting: /usr/bin/python3 /tmp/tmpo5pq5ptj/tmpocjtmzkl.py --working-directory="/tmp/tmpo5pq5ptj" --stage-index=0 --source="/home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-01-15ms.rpt.txt"
jobs submitted
setup complete
stage 0 ready
tessellating roi: 'roi000' (in source '/home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-02-15ms.rpt.txt')...
tessellating roi: 'roi001' (in source '/home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-02-15ms.rpt.txt')...
tessellating roi: 'roi002' (in source '/h

submitting: /usr/bin/python3 /tmp/tmpo5pq5ptj/tmpocjtmzkl.py --working-directory="/tmp/tmpo5pq5ptj" --stage-index=1,2 --source="/home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-02-15ms.rpt.rwa" --region-index=2
submitting: /usr/bin/python3 /tmp/tmpo5pq5ptj/tmpocjtmzkl.py --working-directory="/tmp/tmpo5pq5ptj" --stage-index=1,2 --source="/home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-02-15ms.rpt.rwa" --region-index=3
submitting: /usr/bin/python3 /tmp/tmpo5pq5ptj/tmpocjtmzkl.py --working-directory="/tmp/tmpo5pq5ptj" --stage-index=1,2 --source="/home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-02-15ms.rpt.rwa" --region-index=4
submitting: /usr/bin/python3 /tmp/tmpo5pq5ptj/tmpocjtmzkl.py --working-directory="/tmp/tmpo5pq5ptj" --stage-index=1,2 --source="/home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-02-15ms.rpt.rwa" --region-index=5
submitting: /usr/bin/python3 /tmp/tmpo5pq5ptj/tmpocj

setup complete
stage 2 ready
inferring on roi: 'roi015' (in source '/home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-02-15ms.rpt.rwa')...
stage 2 done
job 15 done

submitting: /usr/bin/python3 /tmp/tmpo5pq5ptj/tmpocjtmzkl.py --working-directory="/tmp/tmpo5pq5ptj" --stage-index=1,2 --source="/home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-02-15ms.rpt.rwa" --region-index=25
setup complete
stage 2 ready
inferring on roi: 'roi016' (in source '/home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-02-15ms.rpt.rwa')...
stage 2 done
job 16 done

submitting: /usr/bin/python3 /tmp/tmpo5pq5ptj/tmpocjtmzkl.py --working-directory="/tmp/tmpo5pq5ptj" --stage-index=1,2 --source="/home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-02-15ms.rpt.rwa" --region-index=26
setup complete
stage 2 ready
inferring on roi: 'roi017' (in source '/home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-0

job 34 done

submitting: /usr/bin/python3 /tmp/tmpo5pq5ptj/tmpocjtmzkl.py --working-directory="/tmp/tmpo5pq5ptj" --stage-index=1,2 --source="/home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-01-15ms.rpt.rwa" --region-index=8
setup complete
stage 2 ready
inferring on roi: 'roi035' (in source '/home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-02-15ms.rpt.rwa')...
stage 2 done
job 35 done

submitting: /usr/bin/python3 /tmp/tmpo5pq5ptj/tmpocjtmzkl.py --working-directory="/tmp/tmpo5pq5ptj" --stage-index=1,2 --source="/home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-01-15ms.rpt.rwa" --region-index=9
setup complete
stage 2 ready
inferring on roi: 'roi000' (in source '/home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-01-15ms.rpt.rwa')...
stage 2 done
job 36 done

submitting: /usr/bin/python3 /tmp/tmpo5pq5ptj/tmpocjtmzkl.py --working-directory="/tmp/tmpo5pq5ptj" --stage-index=1,2 --source="/home

reading file: /tmp/tmpo5pq5ptj/tmpgcve_xng.rwa
for source file: /home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-02-15ms.rpt.txt...
writing file: /home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-02-15ms.rpt.rwa
for source file: /home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-01-15ms.rpt.txt...
writing file: /home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-01-15ms.rpt.rwa
results collected


At this point, the pipeline is complete. As many *.rwa* files as input SPT data files were generated both on the local and remote hosts (if different).

### Sequence diagram

The procedure for distributing computations using the `LocalHost` environment is similar to (slightly simpler than) the `Maestro` environment which is a specialized `SlurmOverSSH` environment:

![Sequence diagram](SlurmOverSSH.svg?1)

The above sequence diagram is available [here](https://sequencediagram.org/index.html#initialData=CoSwLgNgpgBACiADlCIB2soA8oGMCuYIA9mjAO7gAWMAyhPgE4C2A8gG5SO20ASMUNOxCNSzQWADOAKGmIAhoyK4k8tGADmo-IhgBiAMIBWAKIBBACIAhGACpbtfACNm4GJJAATKPbmLlquowAEQAUjoAnmBcADpoAGRxaMTRTsTEANbBMPKSMKF+SiAqCkHBji5uiKK4UJKS2bkwADLSgp6FAaWa2rp6qBpUYE4MsPYA6sSMGVzuXj62ncWBYCEAsrnRjDDJ3o15a0slaqvlDCz7MABKRyshk9Ozu1CX421oHdKhALQAfLbNABc1zqYH8MHi13wZDMEAg0maf1CwLQTlwpE4SgRAB5vt9kTAAKqSWaSXCMJBgaSoDCKBF-NbAgxUPAZGDo9TydCxNDydhciDyJwgVBgCLSNYM4EWYjkNAQYjyTxxEAAMxgrnq6A01O5ihgxFVquk8lwRH50Rgh28pvN8mi8lVWyt9N+jJgBkYUHtsHIUwy2pgnhEeDAU3FiLd0pAkgUYFwNHwJO2AApEBEAJTucmU6R6LTEHT6ABixYADBWyzBVVMBKaaOjmIhiB5LZIwRpYHpSz3i9IC0Wa9tvQmBEbQyBOO4O1BXUCoWQ0il24x5Lp2-JOzA1Tk0OLbZOfS1pDazYeHU7Zq1I+6LDG46PJE57aOU5IqFmyRTEFSad7GK6VzAlc0LuM+8Y0F+uZXH84zAaBSakjmP4qmQHJgtyjBxJQYA0GCkhsooGj4OI6gyH++qGsa4ywcCHBcBS3gGoQiCEHEjDkPI1YilAcZUDING-HBC4wEuUhgKuiBxBuW47mo+5nhasBvKedoXs6byCcJIFkFxHhoBo0AwPhGQmopR5vAOugKsQiCAUyLK4GyG5gEmOJ4kBXy4t885XLxUxUpG87jFyVLtBK3nCXA-ggPIEAwBxXGqjxu6eDAiGMN83jJRgaXonCE4jHUJ4jmpym6rS2yure94vjQahpYw0JxPl0BnqQMBppm2bflSkpRh6xDMMKGBxAoRRxQlnHcdAkhxA17Jej6Y1cB47YSFNSU8eRerbFRZllS6zTee6zQxqshowMgjBrdEQStYVs0lQeSlHVKIISSAUBTnF8UPWeRUyC9R7XidgLPeZlqtOFoTeUCQA).

### Input data files are not dispatched

The above diagram calls for a long explanation but, first of all, the most important point here is that scripts and executables only are dispatched, and the **input data are NOT dispatched** onto the worker side.

This means that the user has to prepare the data on both the submit and worker sides.
This includes SPT data files and ROI files.

These input data files must be located (or reachable) at the same paths on both sides, which is not trivial.
The recommended approach consists of locating the data either from the filesystem root (/) or home/user directory.
The home/user directory path is automatically adjusted from a filesystem to the other.

### The *reload* bootstrap stage

While the `tessellate` and `infer` stages have explicit goals, the `reload` stage might look optional.
Indeed, this stage is optional if no so-called *environments* are defined.
In this case, the stages are sequentialy run in the notebook kernel.

In all the parallelizing settings, the analysis tree is updated by the `tessellate` stage only in the processes that actually ran this stage.
On the local (or submit) side that delegates this stage, the local RWAnalyzer object is not aware of the availability of such an update.
However, this update is required for the next stage, not only for the next stage to be fed with data samples, but also for the submit side to determine the number of tasks and command-line arguments to be passed to each `infer` task.

As a consequence, because the `tessellate` stage has updated the analysis trees in the already-existing or newly-created *.rwa* files, the `spt_data` attribute must be redefined so that it will load the updated *.rwa* files before the `infer` stage is scheduled.

Unlike the `tessellate` and `infer` stages, the `reload` stage must be performed both on the submit and worker sides and is qualified here as a *bootstrap* stage.
Every worker runs this bootstrap stage before running the assigned `infer` task.

The pipeline eventually consists of two effective stages, first `tessellate` and second `reload+infer`, both dispatched onto the remote/worker host, plus `reload` that is also run on the local/submit host.

### Stage granularity

A stage is registered in the RWAnalyzer object using the `pipeline.append_stage` method.
A simple `callable` (Python function) can be passed.

Additional keyword arguments allow to define the granularity of the stage, *i.e.* the level of data representation suitable for distributing the data and running parallel instances of the same stage.

For example, with `granularity='ROI'`, a stage will apply in parallel to each region of interest independently, or more exactly to each *support region* independently.

With `granularity='SPT data'`, a stage will apply in parallel to each SPT data file.

Such control is made possible provided that the stage functions crawl the entire data using the following RWAnalyzer iterators:

* `spt_data` (the attribute itself is iterable),
* `spt_data.as_dataframes`,
* `roi.as_support_regions` (but **NOT** `roi.as_individual_roi`),
* `time.as_time_segments`.

The predefined stages exposed by the `stages` module are shipped as `PipelineStage` objects that readily contain default granularity settings. For example, although the `tessellate` stage processes each support region independently, the granularity is set to `'SPT data'` for performance reasons.

### Parallelism for time segments

Unlike `tessellate`, the `infer` stage does operate at `'ROI'` granularity.

In the case the data are also segmented in time, *e.g.* using a sliding window, and no time regularization is performed, a DV inference (for example) can operate in each time segment independently.

However, the predefined `infer` stage does not split the computation down to the `'time segment'` granularity.
This is possible, writing a modified *infer* procedure that iterates the time segments, generates as many `Maps` objects as time segments, commits these multiple `Maps` objects as analysis artefacts into the analysis tree, using different labels (to be generated with the `time.segment_label` method)... but this is out of the scope of this tutorial.

Instead, we can let the `mapper` attribute **locally** parallelize the computation across the different time segments.
Basically, the predefined `infer` stage will schedule as many tasks as support regions (or SPT data files if no ROI are defined), assign each task to different workers, and then the `mapper` attribute will make each task spawn
multiple processes on the worker node to analyze the different time segments in parallel.

Unlike the old `tramway.helper.inference.infer` function, when time segments are defined and no time regularization is expected, the `mapper` attribute operates any inference function/plugin with default `mapper.cell_sampling='connected'`. This makes the time segments be identified in the global microdomain adjacency matrix (spatio-temporal microdomains are also referred to as 'cells') and individualized into separate connected components of microdomains, so that the defined inference procedure applies separately to each connected component.

Two notes:

* a time segment may result in more than one connected component, if some microdomains are marked as not valid and the area or volume to be mapped is consequently not contiguous;
* an inference function/plugin that requires access to all the time segments at once may also require to disable this behavior with `mapper.cell_sampling=None`.


\[To be continued\]